In [ ]:
from qiskit import *

# The ccrca circuit works for summands of length >= 4

t = True
while t:
    summand_x = list(input('Enter summand x:' ))
    summand_y = list(input('Enter summand y:' ))
    if len(summand_x) == len(summand_y):
        length = len(summand_x)
        t = False
    else:
        print('Inputs must be of equal length.')

w = QuantumRegister(2, 'w')
# quantum register holding the ancilla bits
anc = QuantumRegister(3,'anc')
# register holding the carry bit for ripple-carry-adder
c = QuantumRegister(1,'c')
x = QuantumRegister(length, 'x')
y = QuantumRegister(length, 'y')

# classical register of size = length of summands
out = ClassicalRegister(length,'out') 

# quantum circuit consisting of all quantum and classical registers
qc = QuantumCircuit(w,anc,c,x,y,out)

# initialising the control qubits to 1
qc.x(w[0:2])

# reverse the order of digits
summand_x.reverse()
summand_y.reverse()

# initialisation of x and y values
for a, b, i in zip(summand_x, summand_y, range(length)):
    if a is '1':
        qc.x(x[i])
    if b is '1':
        qc.x(y[i])


# subcircuit defined for the controlled-controlled ripple-carry adder for 2 bits

sw = QuantumRegister(2,'sw')
sa = QuantumRegister(3,'sa')
ss = QuantumRegister(1,'ss')
sx = QuantumRegister(length,'sx')
sy = QuantumRegister(length,'sy')
sc = QuantumCircuit(sw,sa,ss,sx,sy,name='ccrca')

sc.ccx(sw[0],sw[1],sa[0])

for i in range(1,length):
    sc.ccx(sa[0],sx[i],sa[1])
    sc.cx(sa[1],sy[i])
    sc.ccx(sa[0],sx[i],sa[1])

sc.ccx(sa[0],sx[1],sa[1])
sc.cx(sa[1],ss[0])
sc.ccx(sa[0],sx[1],sa[1])

sc.ccx(sa[0],sx[0],sa[1])
sc.ccx(sa[1],sy[0],sa[2])
sc.cx(sa[2],ss[0])
sc.ccx(sa[1],sy[0],sa[2])
sc.ccx(sa[0],sx[0],sa[1])

sc.ccx(sa[0],sx[2],sa[1])
sc.cx(sa[1],sx[1])
sc.ccx(sa[0],sx[2],sa[1])

sc.ccx(sa[0],ss[0],sa[1])
sc.ccx(sa[1],sy[1],sa[2])
sc.cx(sa[2],sx[1])
sc.ccx(sa[1],sy[1],sa[2])
sc.ccx(sa[0],ss[0],sa[1])

sc.ccx(sa[0],sx[3],sa[1])
sc.cx(sa[1],sx[2])
sc.ccx(sa[0],sx[3],sa[1])

for i in range(2,length-2):
    sc.ccx(sa[0],sy[i],sa[1])
    sc.ccx(sa[1],sx[i-1],sa[2])
    sc.cx(sa[2],sx[i])
    sc.ccx(sa[1],sx[i-1],sa[2])
    sc.ccx(sa[0],sy[i],sa[1])
    sc.ccx(sa[0],sx[i+2],sa[1])
    sc.cx(sa[1],sx[i+1])
    sc.ccx(sa[0],sx[i+2],sa[1])
    
sc.ccx(sa[0],sy[length-2],sa[1])
sc.ccx(sa[1],sx[length-3],sa[2])
sc.cx(sa[2],sx[length-2])
sc.ccx(sa[1],sx[length-3],sa[2])
sc.ccx(sa[0],sy[length-2],sa[1])

#sc.ccx(sa[0],sx[length-1],sa[1])
#sc.cx(sa[1],sc[0])
#sc.ccx(sa[0],sx[length-1],sa[1])

for i in range(1,length-1):
    sc.cx(sa[0],sy[i])
    
sc.ccx(sa[0],ss[0],sa[1])
sc.cx(sa[1],sy[1])
sc.ccx(sa[0],ss[0],sa[1])

for i in range(2,length):
    sc.ccx(sa[0],sx[i-1],sa[1])
    sc.cx(sa[1],sy[i])
    sc.ccx(sa[0],sx[i-1],sa[1])
    
sc.ccx(sa[0],sx[length-3],sa[1])
sc.ccx(sa[1],sy[length-2],sa[2])
sc.cx(sa[2],sx[length-2])
sc.ccx(sa[1],sy[length-2],sa[2])
sc.ccx(sa[0],sx[length-3],sa[1])

for i in range(length-3,1,-1):
    sc.ccx(sa[0],sx[i-1],sa[1])
    sc.ccx(sa[1],sy[i],sa[2])
    sc.cx(sa[2],sx[i])
    sc.ccx(sa[1],sy[i],sa[2])
    sc.ccx(sa[0],sx[i-1],sa[1])
    sc.ccx(sa[0],sx[i+2],sa[1])
    sc.cx(sa[1],sx[i+1])
    sc.ccx(sa[0],sx[i+2],sa[1])
    sc.cx(sa[0],sy[i+1])

sc.ccx(sa[0],ss[0],sa[1])
sc.ccx(sa[1],sy[1],sa[2])
sc.cx(sa[2],sx[1])
sc.ccx(sa[1],sy[1],sa[2])
sc.ccx(sa[0],ss[0],sa[1])

sc.ccx(sa[0],sx[3],sa[1])
sc.cx(sa[1],sx[2])
sc.ccx(sa[0],sx[3],sa[1])

sc.cx(sa[0],sy[2])

sc.ccx(sa[0],sx[0],sa[1])
sc.ccx(sa[1],sy[0],sa[2])
sc.cx(sa[2],ss[0])
sc.ccx(sa[1],sy[0],sa[2])
sc.ccx(sa[0],sx[0],sa[1])

sc.ccx(sa[0],sx[2],sa[1])
sc.cx(sa[1],sx[1])
sc.ccx(sa[0],sx[2],sa[1])

sc.cx(sa[0],sy[1])

sc.ccx(sa[0],sx[1],sa[1])
sc.cx(sa[1],ss[0])
sc.ccx(sa[0],sx[1],sa[1])

for i in range(0,length):
    sc.ccx(sa[0],sx[i],sa[1])
    sc.cx(sa[1],sy[i])
    sc.ccx(sa[0],sx[i],sa[1])
    
sc.ccx(sw[0],sw[1],sa[0])

subinst = sc.to_instruction()

arglist = []
for i in range(0,2):
    arglist.append(w[i])
for i in range(0,3):
    arglist.append(anc[i])
arglist.append(c[0])
for i in range(0,length):
    arglist.append(x[i])
for i in range(0,length):
    arglist.append(y[i])

qc.append(subinst,arglist)

qc.measure(y,out)
simulator = Aer.get_backend('qasm_simulator')
result = execute(qc,backend = simulator, shots = 1).result()
counts = result.get_counts()
from qiskit.tools.visualization import plot_histogram
for path in counts:
    print(path)